In [1]:
import importlib
import tiktoken

from torch.utils.data import Dataset, DataLoader

In [2]:
with open("/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/NLP_LLM/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of charecters in the text: ", len(raw_text))
print("First 100 charecters of raw_text:\n", raw_text[:99])

Total number of charecters in the text:  20479
First 100 charecters of raw_text:
 I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [3]:
# The goal is to tokenize the whole 20479 tokens of short story and special charecters into embeddings for pretraining LLM

In [4]:
import re

In [5]:
test_text = "Hello, World. This, is a test."

result = re.split(r"(\s)", test_text)

print(result)

['Hello,', ' ', 'World.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [6]:
result2 = re.split(r"[,.] | \s", test_text)
print(result2)

['Hello', 'World', 'This', 'is a test.']


In [7]:
result3 = re.split(r'([,.]|\s)', test_text)
print(result3)

['Hello', ',', '', ' ', 'World', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [8]:
result = result3

In [9]:
result = [item for item in result if item.split()]

print(result)

['Hello', ',', 'World', '.', 'This', ',', 'is', 'a', 'test', '.']


In [10]:
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
print(result)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'Is', ' ', 'this', '--', '', ' ', 'a', ' ', 'test', '?', '']
['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [11]:
# Now that we have our basic tokenizer ready, lets apply it to our raw_text

preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

print(preprocessed[:30])


['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [12]:
print(len(preprocessed))

4690


In [13]:
# Converting tokens into token ids

all_words = sorted(set(preprocessed))

vocab_size = len(all_words)

print(vocab_size)

1130


In [14]:
vocab = {token:integer for integer, token in enumerate(all_words)}

for i, tup in enumerate(vocab.items()):
    print(tup)
    if i > 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)
('His', 51)


In [15]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.?!_"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        idl = [self.int_to_str[i] for i in ids]
        text = " ".join(idl)
        # Replacing spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [16]:
tokenizer = SimpleTokenizerV1(vocab)

In [17]:
text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""

print("Text: \n", text)

ids = tokenizer.encode(text)
print("Token_ids: \n", ids)

Text: 
 "It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride.
Token_ids: 
 [1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [18]:
text_conv = tokenizer.decode(ids)
print(text_conv)

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [19]:
# Adding special tokens

all_tokens = sorted(set(preprocessed))
all_tokens.extend(["<|endoftoken|>", "<|unk|>"])

vocab = {token:integer for integer, token in enumerate(all_tokens)}
print("Vocab size with special tokens: ", len(vocab))

Vocab size with special tokens:  1132


In [20]:
for i, token in enumerate(list(vocab.items())[-5:]):
    print(token)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftoken|>', 1130)
('<|unk|>', 1131)


In [21]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {integer:token for token, integer in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.?!_"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
        ids = [self.str_to_int[i] for i in preprocessed]
        return ids
    
    def decode(self, ids):
        idl = [self.int_to_str[i] for i in ids]
        text = " ".join(idl)
        # Replacing spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [22]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftoken|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftoken|> In the sunlit terraces of the palace.


In [23]:
tokenizer.encode(text)


[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [24]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftoken|> In the sunlit terraces of the <|unk|>.'

In [25]:
print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.8.0


In [26]:
# The tokenizer below is similar to SimpleTokenizerV2 that we defined earlier with two methods : encode and decode

In [27]:
tokenizer = tiktoken.get_encoding("gpt2")

In [28]:
# From the output of the below code, we can see that endoftext token has a value of 50256
# This means that, the total vocabulary made up of all tokens is around 50k - 51k in numbers
# With this vocab we can relate to all words of english (170000 - 200000 words)
# The vocab size of GPT-2, GPT-3 is 50256, where end of token is the last token

In [29]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [30]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


In [31]:
 # Creating input - output pairs 

In [32]:
# Implement the BPE tokenizer on verdict data

enc_text = tokenizer.encode(raw_text)
print("Token length: ", len(enc_text))

Token length:  5145


In [33]:
len(set(enc_text))

1416

In [34]:
enc_sample = enc_text[50:]

In [35]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print("X: ", x)
print("Y: ", y)

X:  [290, 4920, 2241, 287]
Y:  [4920, 2241, 287, 257]


In [36]:
# creating the sequence input

print("Context target pair: ")
for i in range(1, context_size+1):
    context = x[:i]
    desired = y[i-1]
    print(context, "-->", desired)

Context target pair: 
[290] --> 4920
[290, 4920] --> 2241
[290, 4920, 2241] --> 287
[290, 4920, 2241, 287] --> 257


In [37]:
print("Context target pair: ")
for i in range(1, context_size+1):
    context = x[:i]
    desired = y[i-1]
    print(tokenizer.decode(context), "-->", tokenizer.decode([desired]))

Context target pair: 
 and -->  established
 and established -->  himself
 and established himself -->  in
 and established himself in -->  a


In [38]:
class GPTDatasetV1:
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.output_ids = []

        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        for i in range(0, len(token_ids)-max_length, stride):
            input_chunk = token_ids[i : i+max_length]
            ouput_chunk = token_ids[i+1 : i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.output_ids.append(torch.tensor(ouput_chunk))
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.output_ids[idx]

In [39]:
def create_dataloader_v1(txt, max_length, stride, batch_size, shuffle, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length=max_length, stride=stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        drop_last=drop_last,
        shuffle=shuffle,
        num_workers=num_workers
    )
    return dataloader


In [40]:
import torch

print("torch version: ", torch.__version__)

torch version:  2.5.0+cu124


In [41]:
with open("/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/NLP_LLM/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataloader = create_dataloader_v1(raw_text, max_length=4, stride=1, batch_size=1, drop_last=True, shuffle=True, num_workers=0)

data_iter = iter(dataloader)

first_batch = next(data_iter)
print(first_batch)

[tensor([[ 293,   13, 3363,   11]]), tensor([[  13, 3363,   11,  340]])]


In [42]:
# Let's see how the token embeddings are created

In [43]:
# Initially we only take 6 tokens and vector dimension as 3

vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [44]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [45]:
# Let the vocab words be:

# quick fox is in the house

# token ids for ascending order sorted words

# 4 0 3 2 5 1

# The above embedding matrix is

# 0 - - -
# 1 - - -
# 2 - - -
# 3 - - -
# 4 - - -
# 5 - - -

In [46]:
input_ids = torch.tensor([2, 3, 5, 1])

In [47]:
# Now we will print the vecotor embeddings of these tokens

print("embedding weight matrix for the group of words:\n in is the house", )
print(embedding_layer(input_ids))

embedding weight matrix for the group of words:
 in is the house
tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


In [48]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [49]:
max_length = 4

dataloader = create_dataloader_v1(raw_text, max_length=max_length, stride=max_length, batch_size=8, shuffle=False)

data_iter = iter(dataloader)

inputs, target = next(data_iter)

In [50]:
print("Input: ")
print(inputs)
print("Input shape: ", inputs.shape)

Input: 
tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Input shape:  torch.Size([8, 4])


In [51]:
context_length = max_length

pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [52]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))

In [53]:
print("Shape of pos_embeddings: ", pos_embeddings.shape)

Shape of pos_embeddings:  torch.Size([4, 256])


In [54]:
print(pos_embeddings)

tensor([[ 1.7375, -0.5620, -0.6303,  ..., -0.2277,  1.5748,  1.0345],
        [ 1.6423, -0.7201,  0.2062,  ...,  0.4118,  0.1498, -0.4628],
        [-0.4651, -0.7757,  0.5806,  ...,  1.4335, -0.4963,  0.8579],
        [-0.6754, -0.4628,  1.4323,  ...,  0.8139, -0.7088,  0.4827]],
       grad_fn=<EmbeddingBackward0>)


In [55]:
a = reversed([1, 2, 3])

In [56]:
for i in a:
    print(i)

3
2
1


In [57]:
a = [1, 2, 3]
b = a[::-1]
print(b)

[3, 2, 1]
